Transformer在NLP中的成功是基于其提取“序列元素间语义相关性”的能力 。然而，时间序列数据本身通常缺乏语义，其核心在于建模“连续点集中的时间关系”，其中“顺序本身扮演着最关键的角色” 。这促使研究人员必须重新审视，一种为解决一类问题（NLP）而设计的技术，在应用于另一类本质上不同的问题（TSF）时，其核心机制是否依然适用。

这一观点基于一个关键的对比：在NLP中，句子的语义在词语被重新排列后可能仍然很大程度上得以保留，因为其核心是语义关联 。但对于时间序列数据，如股票价格或电力值，点对点之间几乎没有语义相关性，其主要价值在于连续点集之间的时间变化。因此，数据的“顺序本身扮演着最关键的角色” 。论文表明，位置编码更像是为了强迫一个不适合的模型去解决一个特定问题而使用的“拐杖”。随后的实验将证明，这个“拐杖”在许多情况下并不起作用。

1. 可重叠的 patch 分类 提取有效技术指标 OHLCV 量价技术指标与MLP的等效性

2. 两阶段学习，表征学习，掩码自注意力 预训练Encoder

3. 拼接预测层

4. 全期货市场数据

5. 

1. 可以直接使用日线、周线、月线实现更加科学的池化

